In [ ]:
# important packages
import json
import requests
import pandas as pd
import numpy as np
import time

# Random Stuff to Test How Pulling From Asset API Works

In [ ]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&limit=100' 

url = api_endpoint + api_request

In [ ]:
first_pull = requests.get(url).json()

In [ ]:
first_pull['data'][19]

In [ ]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&page=1500000&limit=1' 

url = api_endpoint + api_request

In [ ]:
test_pull2 = requests.get(url).json()['data']
test_pull2[0]

# Python Script

In [ ]:
# make an empty pandas df
assets_database = pd.DataFrame()
sales_database = pd.DataFrame()

# each page contains one NFT asset - each request will obtain one page
page_counter = 1

# api endpoint for ASSETS
asset_endpoint = 'https://wax.api.atomicassets.io/atomicassets'

# api endpoint for SALES
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'



# PERFORM UNIT TESTING
#for i in range(1):
while True:
    # api key for ASSETS 
    asset_key = '/v1/assets?collection_name=mlb.topps&page={page_num}&limit=1000&order=asc'.format(page_num = page_counter)
    
    # form the asset api to pass into the request
    asset_api = asset_endpoint + asset_key
    
    # pull an asset from the API  
    #Because a KeyError is thrown if the rate limit is passed, the try-except block will see when rate limit is passed
    #and keeps on looping until no more error is thrown.
    try:
        asset_pull = requests.get(asset_api).json()['data']

    except Exception as e:
        time.sleep(1)
        continue
        
    # exit the loop if there are no more assets to pull
    #if len(asset_pull) < 1000:
        ##Uncomment below if we want to continue pulling data once we pulled all currently available data
        #time.sleep(0.45)
        #continue
        #break
        
    # iterate through the array of assets - add each asset to a pandas csv
    
    assets_database = assets_database.append(pd.DataFrame.from_dict(asset_pull))
    
    # itearte through each of the 1000 assets in the pull and get their sales via asset id 
    for j in range(len(asset_pull)):
        # obtain the assets from the api pull request
        asset = asset_pull[j]
        
        # obtain the id of the current asset 
        asset_id = asset['asset_id']
        sales_page=1
        while True:
            # api key for SALES of a given asset
            sales_key = '/v1/sales?collection_whitelist=mlb.topps&page={page_num}&asset_id={ID}&limit=100'.format(ID= asset_id, page_num=sales_page) # see cells below 
    
            # form the sales api to pass into the request
            sales_api = sales_endpoint + sales_key
           
            try:
                 # pull the sales of that ASSET from the API
                sales_pull = requests.get(sales_api).json()['data']
                # ADD TO PANDAS DATABASE
                sales_database=sales_database.append(pd.DataFrame.from_dict(sales_pull))
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
            
            if len(sales_pull) <100:
                break
            sales_page +=1
        
       
    
    # increment the page counter to pull a different asset on the next iteration
    page_counter += 1
        
assets_database

In [ ]:
#Testing uniqueness of data

#assets_database.iloc[1000]
#len(pd.unique(assets_database['asset_id']))
assets_database

# Testing Sales

In [ ]:
# api endpoint
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'

In [ ]:
# add a state for sold, listed, etc.. ask the team about this 
# STATE parameter - NOT NEEDED
sales_key = 'https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666'
url = sales_endpoint + sales_key
url

In [ ]:
x = requests.get('https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666&state=3').json()['data']
x